<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s overal message(disaster or not?):
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [1]:
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-30 12:09:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-30 12:09:24 (131 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-30 12:09:24--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.112, 142.250.81.208, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-08-30 12:09:24 (147 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [6]:
train_data = pd.read_csv('train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
test_data = pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [9]:
random_data = train_data.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
Ladies here's how to recover from a #date you totally BOMBED... according to men http://t.co/c5GGSZUGw1 http://t.co/2PiMg9BIcE : This is NOT Disaster tweet
------------------------------------

tweet number 2 :
Remove the http://t.co/2nS5TfnxpA and Linkury Browser Hijacker http://t.co/W2kVScbTLp http://t.co/tn8o00NrLP : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
OMFG??
Didnt expect Drag Me Down to be the first song Pandora played 

OMFG I SCREAMED SO LOUD
My coworker is scared http://t.co/VzcvAdkcQp : This is a Disaster tweet
------------------------------------

tweet number 4 :
So today I fell off a rock scraped my whole butt and nearly drowned #summer2k15 : This is NOT Disaster tweet
------------------------------------

tweet number 5 :
Iranian warship points weapon at American helicopter... http://t.co/cgFZk8Ha1R : This is a Disaster tweet
------------------------------------



## Extract validation data from train data:

In [10]:
train_data['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [11]:
train_data['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [12]:
train_data, val_data, train_label, val_label = train_test_split(train_data['text'].to_numpy(), train_data['target'].to_numpy(), 
                                                                test_size=0.15, 
                                                                random_state=42)

In [13]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [14]:
sum([len(tweet.split()) for tweet in train_data])

96401

In [15]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [16]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [17]:
text_vectorizer.adapt(train_data)

In [18]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorizer(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  736,    9,    1,    8,  161,  681,    5, 1732,   12,    0,
          0,    0,    0,    0])>

In [19]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorizer(random_tweet))

tweet:  INFO R. CURFEW IN OPER UNTIL 2030 Z. TAXIWAYS FOXTROT 5 &amp; FOXTROT 6 NAVBL. WND: 060/5. EXP INST APCH. RWY 05. DAMP. TMP: 10. QNH: 1028. 


vectorized tweet:  tf.Tensor(
[1331  941  393    4 4765  272 4018 3089 4288 2865  184   34 2865  429
 4829], shape=(15,), dtype=int64)


In [20]:
#unique words
unique_words = text_vectorizer.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['nkulw', 'njenga', 'nj36', 'nittys', 'nitroglycerin', 'nitishkumar', 'nite', 'nissannews', 'nissan', 'niqqa']


## Embedding:

In [119]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim =32, input_length=None)

In [120]:
sample_embed = embedding(text_vectorizer(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[-0.03065368, -0.00160731, -0.00378092, -0.0102633 ,  0.0056429 ,
        -0.01120039, -0.00998706, -0.04122064,  0.00414877, -0.04312373,
        -0.03098418,  0.03872221, -0.04859742,  0.01214924, -0.01979053,
        -0.01954125, -0.04196383, -0.02229447, -0.01900027,  0.00428777,
        -0.01131141, -0.04307914, -0.04558872, -0.02391795, -0.04347851,
         0.00046134,  0.00386969, -0.00011862, -0.02349777,  0.00172243,
        -0.02721879, -0.01300786],
       [ 0.03369469, -0.01236475,  0.04156986,  0.00523894, -0.00994968,
        -0.01541388, -0.00217867, -0.01777349, -0.03675556, -0.04174706,
         0.04363487,  0.00833612,  0.03336246,  0.0166719 ,  0.04852393,
         0.00512532,  0.01739409,  0.02597547, -0.03710799, -0.01361191,
         0.04865995, -0.01827974,  0.00860094, -0.02562027,  0.0104692 ,
         0.02576934,  0.00056911,  0.02250001,  0.02970457,  0.0180905 ,
         0.01309771, -0.00387897],
    

## Analyze with Naive Bayes using TF-IDF:

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [122]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [123]:
val_pred = model_0.predict(val_data)
val_pred

array([0, 0, 0, ..., 1, 0, 0])

In [124]:
val_label

array([1, 0, 1, ..., 1, 0, 0])

In [125]:
accuracy_score(val_label, val_pred)

0.7863397548161121

In [126]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       651
           1       0.85      0.61      0.71       491

    accuracy                           0.79      1142
   macro avg       0.80      0.76      0.77      1142
weighted avg       0.80      0.79      0.78      1142



## Analyze with Vanilla Neural Network:

In [127]:
from helper_functions import create_tensorboard_callback

In [128]:
inputs = tf.keras.layers.Input(shape= (1,), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.keras.layers.Dense(10, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs, name='ANN')

In [129]:
model_1.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 32)            320000    
                                                                 
 global_average_pooling1d_4   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                                 
 dense_9 (Dense)             (None, 1)                 11      

In [130]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [131]:
train_label.shape

(6471,)

In [132]:
train_data.shape

(6471,)

In [154]:
# !rm -rf './tensorboard/ANN'

In [134]:
history_1 = model_1.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220830-121809
Epoch 1/10
203/203 [==============================] - 1s 4ms/step - loss: 0.6364 - accuracy: 0.6557 - val_loss: 0.5562 - val_accuracy: 0.7758
Epoch 2/10
203/203 [==============================] - 1s 6ms/step - loss: 0.4434 - accuracy: 0.8204 - val_loss: 0.4579 - val_accuracy: 0.8021
Epoch 3/10
203/203 [==============================] - 1s 3ms/step - loss: 0.3149 - accuracy: 0.8761 - val_loss: 0.4602 - val_accuracy: 0.7986
Epoch 4/10
203/203 [==============================] - 1s 3ms/step - loss: 0.2401 - accuracy: 0.9087 - val_loss: 0.4912 - val_accuracy: 0.7907
Epoch 5/10
203/203 [==============================] - 1s 3ms/step - loss: 0.1863 - accuracy: 0.9332 - val_loss: 0.5330 - val_accuracy: 0.7767
Epoch 6/10
203/203 [==============================] - 1s 3ms/step - loss: 0.1489 - accuracy: 0.9478 - val_loss: 0.5788 - val_accuracy: 0.7793
Epoch 7/10
203/203 [==============================] - 1s 3ms/step - loss: 0.1202 - 

In [135]:
model_1.evaluate(val_data,val_label)

36/36 [==============================] - 0s 1ms/step - loss: 0.7620 - accuracy: 0.7723


[0.7620260715484619, 0.7723292708396912]

In [136]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [137]:
model_0.predict_proba(tweets)

array([[0.20651409, 0.79348591],
       [0.58784876, 0.41215124],
       [0.70686811, 0.29313189]])

In [138]:
model_0.predict(tweets)

array([1, 0, 0])

In [139]:
model_1.predict(tweets)

array([[0.9929359 ],
       [0.79661655],
       [0.10055774]], dtype=float32)

## Retrieve the trained word embeddings:

In [141]:
weights = model_1.get_layer('embedding_4').get_weights()[0]
weights

array([[-0.03967533,  0.01003422, -0.03398962, ...,  0.2527167 ,
        -0.01202833, -0.03339047],
       [ 0.01940476, -0.03045962, -0.0123709 , ...,  0.31293422,
         0.01446598,  0.00281063],
       [ 0.00396632,  0.03246471,  0.05505633, ...,  0.33442026,
        -0.01857412, -0.01999387],
       ...,
       [ 0.14182653,  0.0884058 ,  0.10687743, ..., -0.08671025,
         0.0817781 ,  0.1151053 ],
       [-0.0792477 , -0.05400255, -0.07878293, ...,  0.12293543,
        -0.05214369, -0.04940571],
       [ 0.00728525, -0.07506175,  0.01403932, ...,  0.0820969 ,
        -0.01809017, -0.05030951]], dtype=float32)

In [143]:
weights.shape

(10000, 32)

In [144]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Then, use [Embedding Projector](http://projector.tensorflow.org/) to Visualize the embeddings

## Analyzie with LSTM model:

In [188]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.LSTM(64,  return_sequences=True)(x)
x = tf.keras.layers.LSTM(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs, name='LSTM')

In [189]:
model_2.summary()

Model: "RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 32)            320000    
                                                                 
 lstm_12 (LSTM)              (None, 15, 64)            24832     
                                                                 
 lstm_13 (LSTM)              (None, 32)                12416     
                                                                 
 dense_16 (Dense)            (None, 16)                528       
                                                               

In [190]:
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [191]:
# !rm -rf './tensorboard/RNN'

In [192]:
history_2 = model_2.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','LSTM')])

Saving TensorBoard log files to: tensorboard/RNN/20220830-133234
Epoch 1/10
203/203 [==============================] - 5s 15ms/step - loss: 0.1093 - accuracy: 0.9679 - val_loss: 1.2629 - val_accuracy: 0.7469
Epoch 2/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0317 - accuracy: 0.9845 - val_loss: 1.6927 - val_accuracy: 0.7391
Epoch 3/10
203/203 [==============================] - 3s 12ms/step - loss: 0.0319 - accuracy: 0.9844 - val_loss: 1.3932 - val_accuracy: 0.7347
Epoch 4/10
203/203 [==============================] - 3s 13ms/step - loss: 0.0307 - accuracy: 0.9853 - val_loss: 1.6676 - val_accuracy: 0.7469
Epoch 5/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0287 - accuracy: 0.9858 - val_loss: 1.6588 - val_accuracy: 0.7443
Epoch 6/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0272 - accuracy: 0.9853 - val_loss: 2.1739 - val_accuracy: 0.7163
Epoch 7/10
203/203 [==============================] - 3s 13ms/step - loss: 0.

In [196]:
model_2.evaluate(val_data,val_label)

36/36 [==============================] - 0s 3ms/step - loss: 1.7510 - accuracy: 0.7303


[1.7510497570037842, 0.7302977442741394]

### custom tweets:

In [197]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [198]:
model_2.predict(tweets)

array([[9.9992967e-01],
       [9.9605358e-01],
       [5.6755543e-04]], dtype=float32)

## Analyzie with GRU model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.GRU(64,  return_sequences=True)(x)
x = tf.keras.layers.GRU(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs, name='GRU')

In [ ]:
model_3.summary()

Model: "RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 32)            320000    
                                                                 
 gru_2 (GRU)                 (None, 15, 64)            18816     
                                                                 
 gru_3 (GRU)                 (None, 32)                9408      
                                                                 
 dense_20 (Dense)            (None, 16)                528       
                                                               

In [ ]:
model_3.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_3 = model_3.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','GRU')])

Saving TensorBoard log files to: tensorboard/RNN/20220830-135100
Epoch 1/10
203/203 [==============================] - 6s 16ms/step - loss: 0.1485 - accuracy: 0.9369 - val_loss: 1.0379 - val_accuracy: 0.7478
Epoch 2/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0457 - accuracy: 0.9828 - val_loss: 1.1582 - val_accuracy: 0.7154
Epoch 3/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0312 - accuracy: 0.9836 - val_loss: 1.2814 - val_accuracy: 0.7250
Epoch 4/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0333 - accuracy: 0.9839 - val_loss: 0.9270 - val_accuracy: 0.7644
Epoch 5/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0325 - accuracy: 0.9844 - val_loss: 1.5189 - val_accuracy: 0.7364
Epoch 6/10
203/203 [==============================] - 2s 12ms/step - loss: 0.0273 - accuracy: 0.9852 - val_loss: 1.7820 - val_accuracy: 0.7522
Epoch 7/10
203/203 [==============================] - 2s 12ms/step - loss: 0.

In [ ]:
model_3.evaluate(val_data,val_label)

36/36 [==============================] - 0s 3ms/step - loss: 1.8151 - accuracy: 0.7504


[1.8151158094406128, 0.75043785572052]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_3.predict(tweets)

array([[9.9998200e-01],
       [9.9980426e-01],
       [2.8124452e-04]], dtype=float32)

## Analyzie with Bidirectional LSTM model:

In [212]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)


x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

In [213]:
model_4.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 32)            320000    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                

In [214]:
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [215]:
# !rm -rf './tensorboard/RNN'

In [216]:
history_4 = model_4.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/RNN/20220830-142613
Epoch 1/10
203/203 [==============================] - 9s 24ms/step - loss: 0.0884 - accuracy: 0.9734 - val_loss: 1.2831 - val_accuracy: 0.7557
Epoch 2/10
203/203 [==============================] - 4s 19ms/step - loss: 0.0319 - accuracy: 0.9845 - val_loss: 1.5075 - val_accuracy: 0.7496
Epoch 3/10
203/203 [==============================] - 4s 19ms/step - loss: 0.0282 - accuracy: 0.9849 - val_loss: 1.8980 - val_accuracy: 0.7399
Epoch 4/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0264 - accuracy: 0.9850 - val_loss: 2.0657 - val_accuracy: 0.7364
Epoch 5/10
203/203 [==============================] - 4s 20ms/step - loss: 0.0302 - accuracy: 0.9841 - val_loss: 1.4312 - val_accuracy: 0.7023
Epoch 6/10
203/203 [==============================] - 5s 24ms/step - loss: 0.0315 - accuracy: 0.9849 - val_loss: 1.6043 - val_accuracy: 0.7478
Epoch 7/10
203/203 [==============================] - 4s 22ms/step - loss: 0.

In [217]:
model_4.evaluate(val_data,val_label)

36/36 [==============================] - 0s 5ms/step - loss: 1.9325 - accuracy: 0.7426


[1.9324778318405151, 0.7425569295883179]

### custom tweets:

In [218]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [219]:
model_4.predict(tweets)

array([[9.9996984e-01],
       [9.9641371e-01],
       [3.7804246e-04]], dtype=float32)